# Latent Dirichlet Allocation

## Data

We will be using articles from NPR (National Public Radio), obtained from their website [www.npr.org](http://www.npr.org)

In [1]:
import pandas as pd

In [2]:
npr = pd.read_csv('npr.csv')

In [3]:
npr.head()

Article
0  In the Washington of 2016, even when the polic...
1    Donald Trump has used Twitter  —   his prefe...
2    Donald Trump is unabashedly praising Russian...
3  Updated at 2:50 p. m. ET, Russian President Vl...
4  From photography, illustration and video, to d...

Notice how we don't have the topic of the articles! Let's use LDA to attempt to figure out clusters of the articles.

## Preprocessing

In [4]:
from sklearn.feature_extraction.text import CountVectorizer

**`max_df`**` : float in range [0.0, 1.0] or int, default=1.0`<br>
When building the vocabulary ignore terms that have a document frequency strictly higher than the given threshold (corpus-specific stop words). If float, the parameter represents a proportion of documents, integer absolute counts. This parameter is ignored if vocabulary is not None.

**`min_df`**` : float in range [0.0, 1.0] or int, default=1`<br>
When building the vocabulary ignore terms that have a document frequency strictly lower than the given threshold. This value is also called cut-off in the literature. If float, the parameter represents a proportion of documents, integer absolute counts. This parameter is ignored if vocabulary is not None.

max_df=0.95: This sets the threshold for ignoring terms that appear in more than 95% of the documents. It helps to remove common terms that are not likely to be meaningful for your analysis.

min_df=2: This sets the threshold for ignoring terms that appear in fewer than 2 documents. It helps to remove rare terms that might not contribute much to the overall analysis.

stop_words='english': This specifies that common English stop words (like "and", "is", "in", etc.) should be removed from the documents. Stop words are typically removed because they occur frequently and do not usually contain significant information for the model.


In [5]:
cv = CountVectorizer(max_df=0.95, min_df=2, stop_words='english')

In [6]:
dtm = cv.fit_transform(npr['Article'])

In [7]:
dtm

<Compressed Sparse Row sparse matrix of dtype 'int64'
	with 3033388 stored elements and shape (11992, 54777)>

## LDA

In [8]:
from sklearn.decomposition import LatentDirichletAllocation

LatentDirichletAllocation is a class in the sklearn.decomposition module in Python's scikit-learn library, used for topic modeling. It’s a machine learning technique to discover hidden (latent) topics in a set of documents.

Here’s what the parameters mean:

n_components=7: This sets the number of topics that the model will try to find in the data. In this case, the model will identify 7 different topics.

random_state=42: This ensures reproducibility. The random state is like a seed for the random number generator. Setting it to 42 ensures that you get the same results each time you run the model with the same data and parameters.

When you create this LatentDirichletAllocation object, you’re setting up a model that will look for 7 distinct topics in your text data and will do so in a reproducible way thanks to the fixed random state.

The number of documents you use is small for the method (it works much better when trained on a data source of the size of Wikipedia). Therefore the results will be rather crude and you have to be aware of that. This is why you should not aim for a large number of topics (you chose 10 which could maybe go sensibly up to 20 in your case).

As for the other parameters:

random_state - this serves as a seed (in case you wanted to repeat exactly the training process)

chunksize - number of documents to consider at once (affects the memory consumption)

update_every - update the model every update_every chunksize chunks (essentially, this is for memory consumption optimization)

passes - how many times the algorithm is supposed to pass over the whole corpus

alpha - to cite the documentation:

can be set to an explicit array = prior of your choice. It also support special values of `‘asymmetric’ and ‘auto’: the former uses a fixed normalized asymmetric 1.0/topicno prior, the latter learns an asymmetric prior directly from your data.

per_word_topics - setting this to True allows for extraction of the most likely topics given a word. The training process is set in such a way that every word will be assigned to a topic. Otherwise, words that are not indicative are going to be omitted. phi_value is another parameter that steers this process - it is a threshold for a word treated as indicative or not.

https://stackoverflow.com/questions/50805556/understanding-parameters-in-gensim-lda-model

In [9]:
LDA = LatentDirichletAllocation(n_components=7,random_state=42)

In [10]:
# This can take awhile, we're dealing with a large amount of documents!
LDA.fit(dtm)

LatentDirichletAllocation(n_components=7, random_state=42)

## Showing Stored Words

In [12]:
len(cv.get_feature_names_out())

54777

In [13]:
import random

In [15]:
for i in range(10):
    random_word_id = random.randint(0,54776)
    print(cv.get_feature_names_out()[random_word_id])

grube
beefsteaks
decatur
mandarins
wallpaper
changer
sustaining
decade
likeness
mains


In [17]:
for i in range(10):
    random_word_id = random.randint(0,54776)
    print(cv.get_feature_names_out()[random_word_id])

tethered
swans
listener
nanograms
vanishes
autism
orbit
neurologist
asses
karlan


### Showing Top Words Per Topic

In [18]:
len(LDA.components_)

7

In [19]:
LDA.components_

array([[8.64332806e+00, 2.38014333e+03, 1.42900522e-01, ...,
        1.43006821e-01, 1.42902042e-01, 1.42861626e-01],
       [2.76191749e+01, 5.36394437e+02, 1.42857148e-01, ...,
        1.42861973e-01, 1.42857147e-01, 1.42906875e-01],
       [7.22783888e+00, 8.24033986e+02, 1.42857148e-01, ...,
        6.14236247e+00, 2.14061364e+00, 1.42923753e-01],
       ...,
       [3.11488651e+00, 3.50409655e+02, 1.42857147e-01, ...,
        1.42859912e-01, 1.42857146e-01, 1.42866614e-01],
       [4.61486388e+01, 5.14408600e+01, 3.14281373e+00, ...,
        1.43107628e-01, 1.43902481e-01, 2.14271779e+00],
       [4.93991422e-01, 4.18841042e+02, 1.42857151e-01, ...,
        1.42857146e-01, 1.43760101e-01, 1.42866201e-01]])

In [20]:
len(LDA.components_[0])

54777

In [21]:
single_topic = LDA.components_[0]

In [22]:
# Returns the indices that would sort this array.
single_topic.argsort()

array([ 2475, 18302, 35285, ..., 22673, 42561, 42993])

In [23]:
# Word least representative of this topic
single_topic[18302]

np.float64(0.14285714309286987)

In [24]:
# Word most representative of this topic
single_topic[42993]

np.float64(6247.245510521025)

In [25]:
# Top 10 words for this topic:
single_topic.argsort()[-10:]

array([33390, 36310, 21228, 10425, 31464,  8149, 36283, 22673, 42561,
       42993])

In [26]:
top_word_indices = single_topic.argsort()[-10:]

In [28]:
for index in top_word_indices:
    print(cv.get_feature_names_out()[index])

new
percent
government
company
million
care
people
health
said
says


These look like business articles perhaps... Let's confirm by using .transform() on our vectorized articles to attach a label number. But first, let's view all the 10 topics found.

In [29]:
for index,topic in enumerate(LDA.components_):
    print(f'THE TOP 15 WORDS FOR TOPIC #{index}')
    print([cv.get_feature_names_out()[i] for i in topic.argsort()[-15:]])
    print('\n')

THE TOP 15 WORDS FOR TOPIC #0
['companies', 'money', 'year', 'federal', '000', 'new', 'percent', 'government', 'company', 'million', 'care', 'people', 'health', 'said', 'says']


THE TOP 15 WORDS FOR TOPIC #1
['military', 'house', 'security', 'russia', 'government', 'npr', 'reports', 'says', 'news', 'people', 'told', 'police', 'president', 'trump', 'said']


THE TOP 15 WORDS FOR TOPIC #2
['way', 'world', 'family', 'home', 'day', 'time', 'water', 'city', 'new', 'years', 'food', 'just', 'people', 'like', 'says']


THE TOP 15 WORDS FOR TOPIC #3
['time', 'new', 'don', 'years', 'medical', 'disease', 'patients', 'just', 'children', 'study', 'like', 'women', 'health', 'people', 'says']


THE TOP 15 WORDS FOR TOPIC #4
['voters', 'vote', 'election', 'party', 'new', 'obama', 'court', 'republican', 'campaign', 'people', 'state', 'president', 'clinton', 'said', 'trump']


THE TOP 15 WORDS FOR TOPIC #5
['years', 'going', 've', 'life', 'don', 'new', 'way', 'music', 'really', 'time', 'know', 'think',

### Attaching Discovered Topic Labels to Original Articles

In [30]:
dtm

<Compressed Sparse Row sparse matrix of dtype 'int64'
	with 3033388 stored elements and shape (11992, 54777)>

In [31]:
dtm.shape

(11992, 54777)

In [32]:
len(npr)

11992

In [33]:
topic_results = LDA.transform(dtm)

In [34]:
topic_results.shape

(11992, 7)

In [35]:
topic_results[0]

array([1.61040465e-02, 6.83341493e-01, 2.25376318e-04, 2.25369288e-04,
       2.99652737e-01, 2.25479379e-04, 2.25497980e-04])

In [36]:
topic_results[0].round(2)

array([0.02, 0.68, 0.  , 0.  , 0.3 , 0.  , 0.  ])

In [37]:
topic_results[0].argmax()

np.int64(1)

This means that our model thinks that the first article belongs to topic #1.

### Combining with Original Data

In [38]:
npr.head()

Article
0  In the Washington of 2016, even when the polic...
1    Donald Trump has used Twitter  —   his prefe...
2    Donald Trump is unabashedly praising Russian...
3  Updated at 2:50 p. m. ET, Russian President Vl...
4  From photography, illustration and video, to d...

In [39]:
topic_results.argmax(axis=1)

array([1, 1, 1, ..., 3, 4, 0])

In [40]:
npr['Topic'] = topic_results.argmax(axis=1)

In [41]:
npr.head(10)

Article  Topic
0  In the Washington of 2016, even when the polic...      1
1    Donald Trump has used Twitter  —   his prefe...      1
2    Donald Trump is unabashedly praising Russian...      1
3  Updated at 2:50 p. m. ET, Russian President Vl...      1
4  From photography, illustration and video, to d...      2
5  I did not want to join yoga class. I hated tho...      3
6  With a   who has publicly supported the debunk...      3
7  I was standing by the airport exit, debating w...      2
8  If movies were trying to be more realistic, pe...      3
9  Eighteen years ago, on New Year’s Eve, David F...      2

## Great work!